# Imports

Clone and install https://github.com/MTG/pycompmusic

In [ ]:
# token = "185d809a541e65d254bc577774a3fce1749bbf50"

In [ ]:
# import compmusic

# from compmusic import dunya as dn
# from compmusic.dunya import hindustani as hi
# from compmusic.dunya import carnatic as ca
# from compmusic.dunya import docserver as ds
# from compmusic import musicbrainz

# dn.set_token(token)

Download data using `Ismir2018TutorialNotebooks/notebooks/downloadAllSARAGAContent.ipynb`

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import librosa
from IPython.display import Audio as ipy_audio

# Data

In [ ]:
data_dir = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-freqs-from-dunya/"

In [ ]:
track = "Omkar Dadarkar - Raag Bhimpalasi"

In [ ]:
audio_file = track + ".mp3"
metadata_file = track + ".json"
tonic_file = track + ".ctonic.txt"
pitch_file = track + ".pitch.txt"

Metadata + Tonic

In [ ]:
with open(data_dir + metadata_file) as json_data:
    metadata = json.load(json_data)

tonic = float(np.loadtxt(data_dir + tonic_file))

In [ ]:
print(f"length = {metadata['length']/1000} seconds")
print(f"base tonic = {tonic}")

Audio File

In [ ]:
y, sr = librosa.load(data_dir + audio_file)

In [ ]:
## Spectrogram

# D = librosa.stft(y)
# S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

# plt.figure().set_figwidth(12)
# librosa.display.specshow(S_db, x_axis="time", y_axis="hz", bins_per_octave=22)
# plt.colorbar()

Pitch Annotations

In [ ]:
pitch_annotations = []
with open(data_dir + pitch_file) as pf:
    for line in pf:
        pitch_annotations.append(line.split())
pitch_annotations = np.array(pitch_annotations).astype(float)

In [ ]:
print(f"{pitch_annotations.shape[0]/metadata['length']*1000} annotations per second")

# Stuff

Pitch Histogram

In [ ]:
nz_annotations = pitch_annotations[np.where(pitch_annotations[:,1] != 0), 1][0]
h = plt.hist(nz_annotations/tonic, bins=700)
plt.xscale("log")
_ = plt.xlim(0.5, 3)

counts = h[0]
notes = (h[1][:-1]+h[1][1:])/2

In [ ]:
fig = px.line(
    pd.DataFrame({"note": notes, "count": counts}),
    x="note", y="count",
    log_x=True
)
# fig.update_layout(xaxis_range=[0.5, 3.1])
fig.update_traces(hovertemplate="freq_ratio: %{x}<br>occurence: %{y}")
fig.show()

In [ ]:
print(track)

Manual Peak Finding: Add to the list Below

In [ ]:
raag = "Bhimpalasi"  # Omkar Dadarkar
swar = {
    "n.": 0.888, 
    "S": 1.002,
    "R": 1.1225,
    "g": 1.189,  # may be higher
    "M": 1.333,
    "P": 1.495,  # may be higher
    "D": 1.689,  # may be lower
    "n": 1.784,  # may be lower
    "S^": 2.00,
    "R^": 2.37,
}